In [20]:
import pandas as pd
import numpy as np
import pulp

# Load data from CSV file
df = pd.read_csv("LnT Kalyani Input.csv")

# Define the number of rebars and their lengths
num_rebars = len(df)
rebar_lengths = df["Length"].tolist()

# Define the number of pieces required
demand = df["Number"].sum()

# Define the solver
prob = pulp.LpProblem("Rebar Cutting Problem", pulp.LpMaximize)

# Define the variables
cut_vars = pulp.LpVariable.dicts("CutVar_%d" % i, range(num_rebars), lowBound=0, cat="Integer")
pattern_vars = pulp.LpVariable.dicts("PatternVar_%d" % i, [], lowBound=0, cat="Continuous")

# Define the objective function
prob += pulp.lpSum(cut_vars)

# Define the constraints
for i in range(num_rebars):
    prob += cut_vars[i] <= rebar_lengths[i]
    prob.constraints[prob.constraints.get("CutVar_%d" % i, None)] = cut_vars[i] <= rebar_lengths[i]
    prob.constraints["CutVar_%d" % i].name = "CutVar_%d" % i

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = np.random.randint(1, high=num_rebars, size=demand)
    pattern_length = sum(rebar_lengths[i-1] for i in pattern if i > 0)

    # Calculate the coefficient of the new variable
    coeffs = [0] * num_rebars
    for i in pattern:
        coeffs[i-1] += 1

    # Check if the new pattern is profitable
    reduced_cost = sum(coeffs[i-1] * dual_values[i] for i in range(num_rebars)) - pattern_length
    if reduced_cost > 0:
        # Add the new cutting pattern to the problem as a new variable
        pattern_var = pulp.LpVariable("PatternVar_%d" % len(pattern_vars), lowBound=0, cat="Continuous")
        pattern_vars[len(pattern_vars)] = pattern_var
        prob += pattern_var == pattern_length
        for i in pattern:
            prob += cut_vars[i-1] >= pattern_var / rebar_lengths[i-1]
            prob.constraints["CutVar_%d" % (i-1)].name = "CutVar_%d" % (i-1)
        return pattern_var
    else:
        return None

# Define the column generation loop
while True:
    # Solve the linear programming problem
    prob.solve()

    # Check if the problem is infeasible
    if pulp.LpStatus[prob.status] != "Optimal":
        break

    # Get the dual values of the constraints
    dual_values = [prob.constraints[i].pi for i in range(num_rebars)]

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Check if no new pattern was found
    if new_pattern is None:
        break

# Print the results
print("Number of rebars:", num_rebars)
print("Rebar lengths:", rebar_lengths)
print("Number of pieces required:", demand)
print("Number of cutting patterns:", len(pattern_vars))
print("Total yield:", pulp.value(prob.objective))

In [2]:
import csv
import gurobipy as gp

# Define the input data
data = []
with open('LnT Kalyani Input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'diameter': int(row['Diameter']), 'length': float(row['Length']), 'number': int(row['Number']), 'bend': int(row['Bend'])})

# Define the rebar length and the number of pieces required
rebar_length = 12
num_pieces = sum(d['number'] for d in data)

# Define the solver
m = gp.Model('rebar_cutting')

# Define the variables
cut_vars = m.addVars(len(data), vtype=gp.GRB.INTEGER, name='cut_vars')

# Define the objective function
m.setObjective(gp.quicksum(cut_vars[i] for i in range(len(data))), gp.GRB.MAXIMIZE)

# Define the constraints
for i in range(len(data)):
    m.addConstr(cut_vars[i] <= data[i]['length'] * data[i]['number'])
    m.addConstr(cut_vars[i] >= 0)
    m.addConstr(cut_vars[i] <= rebar_length)
    m.addConstr(cut_vars[i] >= rebar_length * (1 - (1 - data[i]['length']) / rebar_length) * data[i]['number'])

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = []
    for d in data:
        if d['length'] < rebar_length and d['number'] > 0:
            num_pieces_to_cut = min(d['number'], int(rebar_length / d['length']))
            pattern.append((d['length'] * num_pieces_to_cut, d['length'], num_pieces_to_cut))
            d['number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    # Optimize the current LP relaxation
    m.optimize()

    # Check if the solution is integer
    if m.cbGet(gp.GRB.CB_STATUS) == gp.GRB.OPTIMAL:
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = m.addVars(len(new_pattern), vtype=gp.GRB.INTEGER, name='new_vars')
        m.update()
        for j in range(len(new_pattern)):
            length, pattern_length, num_pieces = new_pattern[j]
            m.addConstr(new_vars[j] == num_pieces)
            m.addConstr(new_vars[j] <= cut_vars[data.index({'diameter': length, 'length': pattern_length, 'number': 1, 'bend': 0})])
            m.addConstr(new_vars[j] <= rebar_length)
            m.addConstr(new_vars[j] >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            m.addConstr(new_vars[j] >= 0)
            m.addConstr(cut_vars[data.index({'diameter': length, 'length': pattern_length, 'number': 1, 'bend': 0})] >= new_vars[j])

# Print the results
print('Number of pieces required:', num_pieces)
print('Number of cuts:', m.objVal)
print('Yield:', m.objVal / num_pieces)

In [3]:
import csv
import libmamba

# Define the input data
data = []
with open('data.csv') as csvfile:
    reader = csv.DictReader("LnT Kalyani Input.csv")
    for row in reader:
        data.append({'diameter': int(row['Diameter']), 'length': float(row['Length']), 'number': int(row['Number']), 'bend': int(row['Bend'])})

# Define the rebar length and the number of pieces required
rebar_length = 12
num_pieces = sum(d['number'] for d in data)

# Define the solver
solver = libmamba.Solver()

# Define the variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.variable(name="CutVar_%d" % i, lb=0, ub=d['number'])

# Define the objective function
solver.objective(solver.maximize, sum([cut_vars[i] for i in cut_vars]))

# Define the constraints
for i, d in enumerate(data):
    solver.constraint(cut_vars[i] * d['length'] <= rebar_length)

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = []
    for d in data:
        if d['length'] < rebar_length and d['number'] > 0:
            num_pieces_to_cut = min(d['number'], int(rebar_length / d['length']))
            pattern.append((d['length'] * num_pieces_to_cut, d['length'], num_pieces_to_cut))
            d['number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    # Solve the current LP relaxation
    solver.solve()

    # Check if the solution is integer
    if solver.primal_integrality() <= 1e-4:
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j in range(len(new_pattern)):
            length, pattern_length, num_pieces = new_pattern[j]
            new_var = solver.variable(name="NewVar_%d" % j, lb=0, ub=num_pieces)
            new_vars.append(new_var)
            solver.constraint(new_var == num_pieces)
            solver.constraint(new_var * pattern_length <= rebar_length)
            solver.constraint(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.constraint(new_var >= 0)
            solver.constraint(cut_vars[data.index({'diameter': length, 'length': pattern_length, 'number': 1, 'bend': 0})] >= new_var)

# Print the results
print('Number of pieces required:', num_pieces)
print('Number of cuts:', solver.objective_value())
print('Yield:', solver.objective_value() / num_pieces)

# Plot the new cutting patterns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 6))
for i, d in enumerate(data):
    ax.barh(y=i, width=d['length'] * cut_vars[i].solution_value(), height=0.5, left=0, label=f'Diameter {d["diameter"]}')
ax.set_xlabel('Length (m)')
ax.set_ylabel('Rebar')
ax.set_title('Cut and Bent (CAB) Rebar Optimization Model')
ax.legend()

In [7]:
import csv
from ortools.linear_solver import pywraplp

# Define the input data
data = [
    {'Diameter': 12, 'Length': 12, 'Number': 3976},
    {'Diameter': 12, 'Length': 5.405, 'Number': 142},
    {'Diameter': 12, 'Length': 6, 'Number': 142},
    {'Diameter': 12, 'Length': 6.82, 'Number': 142},
    {'Diameter': 12, 'Length': 7.26, 'Number': 142},
    {'Diameter': 12, 'Length': 8.245, 'Number': 142},
    {'Diameter': 12, 'Length': 9.49, 'Number': 142},
    {'Diameter': 12, 'Length': 9.66, 'Number': 142},
    {'Diameter': 12, 'Length': 11.08, 'Number': 142},
    {'Diameter': 12, 'Length': 11.265, 'Number': 142},
    {'Diameter': 12, 'Length': 11.68, 'Number': 142},
    {'Diameter': 12, 'Length': 11.735, 'Number': 142},
    {'Diameter': 12, 'Length': 11.79, 'Number': 142},
    {'Diameter': 12, 'Length': 11.845, 'Number': 142},
    {'Diameter': 12, 'Length': 11.895, 'Number': 142},
    {'Diameter': 12, 'Length': 11.95, 'Number': 142},
    {'Diameter': 12, 'Length': 5.955, 'Number': 426},
    {'Diameter': 12, 'Length': 7.575, 'Number': 426},
    {'Diameter': 16, 'Length': 3.885, 'Number': 748},
    {'Diameter': 12, 'Length': 6.205, 'Number': 426},
    {'Diameter': 12, 'Length': 7.57, 'Number': 426},
    {'Diameter': 12, 'Length': 8.935, 'Number': 426},
    {'Diameter': 12, 'Length': 9.22, 'Number': 1278},
    {'Diameter': 12, 'Length': 8.87, 'Number': 576},
    {'Diameter': 12, 'Length': 8.77, 'Number': 1128},
    {'Diameter': 12, 'Length': 7.14, 'Number': 376},
    {'Diameter': 12, 'Length': 7.475, 'Number': 376},
    {'Diameter': 12, 'Length': 7.935, 'Number': 376},
    {'Diameter': 12, 'Length': 8.27, 'Number': 376},
    {'Diameter': 12, 'Length': 8.615, 'Number': 376},
    {'Diameter': 12, 'Length': 7.22, 'Number': 192},
    {'Diameter': 12, 'Length': 7.54, 'Number': 192},
    {'Diameter': 12, 'Length': 7.97, 'Number': 192},
    {'Diameter': 12, 'Length': 8.29, 'Number': 192},
    {'Diameter': 12, 'Length': 8.73, 'Number': 192},
   ]
with open('LnT Kalyani Input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the rebar length and the number of pieces required
rebar_length = 12
num_pieces = sum(d['Number'] for d in data)

# Define the solver
solver = pywraplp.Solver('CAB Rebar Optimization Model', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.IntVar(0, d['Number'], 'CutVar_%d' % i)

# Define the objective function
solver.Maximize(solver.Sum([cut_vars[i] for i in cut_vars]))

# Define the constraints
for i, d in enumerate(data):
    solver.Add(cut_vars[i] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = []
    for d in data:
        if d['Length'] < rebar_length and d['Number'] > 0:
            num_pieces_to_cut = min(d['Number'], int(rebar_length / d['Length']))
            pattern.append((d['Length'] * num_pieces_to_cut, d['Length'], num_pieces_to_cut))
            d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    # Solve the current LP relaxation
    solver.Solve()

    # Check if the solution is integer
    if solver.Integer():
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j in range(len(new_pattern)):
            length, pattern_length, num_pieces = new_pattern[j]
            new_var = solver.IntVar(0, num_pieces, 'NewVar_%d' % j)
            new_vars.append(new_var)
            solver.Add(new_var == num_pieces)
            solver.Add(new_var * pattern_length <= rebar_length)
            solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.Add(new_var >= 0)
            solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})] >= new_var)

# Print the results
print('Number of pieces required:', num_pieces)
print('Number of cuts:', solver.Objective().Value())
print('Yield:', solver.Objective().Value() / num_pieces)

# Plot the new cutting patterns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 6))
for i, d in enumerate(data):
    ax.barh(y=i, width=d['Length'] * cut_vars[i].solution_value(), height=0.5, left=0, label=f'Diameter {d["Diameter"]}')
ax.set_xlabel('Length (m)')
ax.set_ylabel('Rebar')
ax.set_title('Cut and Bent (CAB) Rebar Optimization Model')
ax.leg

In [12]:
import csv
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

# Define the input data
data = [{'Diameter': 12, 'Length': 12, 'Number': 3976},
        {'Diameter': 12, 'Length': 5.405, 'Number': 142},
        {'Diameter': 12, 'Length': 6, 'Number': 142},
        {'Diameter': 12, 'Length': 6.82, 'Number': 142},
        {'Diameter': 12, 'Length': 7.26, 'Number': 142},
        {'Diameter': 12, 'Length': 8.245, 'Number': 142},
        {'Diameter': 12, 'Length': 9.49, 'Number': 142},
        {'Diameter': 12, 'Length': 9.66, 'Number': 142},
        {'Diameter': 12, 'Length': 11.08, 'Number': 142},
        {'Diameter': 12, 'Length': 11.265, 'Number': 142},
        {'Diameter': 12, 'Length': 11.68, 'Number': 142},
        {'Diameter': 12, 'Length': 11.735, 'Number': 142},
        {'Diameter': 12, 'Length': 11.79, 'Number': 142},
        {'Diameter': 12, 'Length': 11.845, 'Number': 142},
        {'Diameter': 12, 'Length': 11.895, 'Number': 142},
        {'Diameter': 12, 'Length': 11.95, 'Number': 142},
        {'Diameter': 12, 'Length': 5.955, 'Number': 426},
        {'Diameter': 12, 'Length': 7.575, 'Number': 426},
        {'Diameter': 16, 'Length': 3.885, 'Number': 748},
        {'Diameter': 12, 'Length': 6.205, 'Number': 426},
        {'Diameter': 12, 'Length': 7.57, 'Number': 426},
        {'Diameter': 12, 'Length': 8.935, 'Number': 426},
        {'Diameter': 12, 'Length': 9.22, 'Number': 1278},
        {'Diameter': 12, 'Length': 8.87, 'Number': 576},
        {'Diameter': 12, 'Length': 8.77, 'Number': 1128},
        {'Diameter': 12, 'Length': 7.14, 'Number': 376},
        {'Diameter': 12, 'Length': 7.475, 'Number': 376},
        {'Diameter': 12, 'Length': 7.935, 'Number': 376},
        {'Diameter': 12, 'Length': 8.27, 'Number': 376},
        {'Diameter': 12, 'Length': 8.615, 'Number': 376},
        {'Diameter': 12, 'Length': 7.22, 'Number': 192},
        {'Diameter': 12, 'Length': 7.54, 'Number': 192},
        {'Diameter': 12, 'Length': 7.97, 'Number': 192},
        {'Diameter': 12, 'Length': 8.29, 'Number': 192},
        {'Diameter': 12, 'Length': 8.73, 'Number': 192}]

with open('LnT Kalyani Input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the rebar length and the number of pieces required
rebar_length = 12
num_pieces = sum(d['Number'] for d in data)

# Define the solver
solver = pywraplp.Solver('CAB Rebar Optimization Model', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.IntVar(0, d['Number'], f'CutVar_{i}')

# Define the objective function
solver.Maximize(solver.Sum([cut_vars[i] for i in cut_vars]))

# Define the constraints
for i, d in enumerate(data):
    solver.Add(cut_vars[i] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = []
    for d in data:
        if d['Length'] < rebar_length and d['Number'] > 0:
            num_pieces_to_cut = min(d['Number'], int(rebar_length / d['Length']))
            pattern.append((d['Length'] * num_pieces_to_cut, d['Length'], num_pieces_to_cut))
            d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    # Solve the current LP relaxation
    solver.Solve()

    # Check if the solution is integer
    if all(cut_vars[i].solution_value() % 1 == 0 for i in cut_vars):
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j, (length, pattern_length, num_pieces) in enumerate(new_pattern):
            new_var = solver.IntVar(0, num_pieces, f'NewVar_{j}')
            new_vars.append(new_var)
            solver.Add(new_var == num_pieces)
            solver.Add(new_var * pattern_length <= rebar_length)
            solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.Add(new_var >= 0)
            solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})] >= new_var)

# Print the results
print('Number of pieces required:', num_pieces)
print('Number of cuts:', solver.Objective().Value())
print('Yield:', solver.Objective().Value() / num_pieces)


In [14]:
import csv
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

# Define the input data
data = [{'Diameter': 12, 'Length': 12, 'Number': 3976},
        {'Diameter': 12, 'Length': 5.405, 'Number': 142},
        {'Diameter': 12, 'Length': 6, 'Number': 142},
        {'Diameter': 12, 'Length': 6.82, 'Number': 142},
        {'Diameter': 12, 'Length': 7.26, 'Number': 142},
        {'Diameter': 12, 'Length': 8.245, 'Number': 142},
        {'Diameter': 12, 'Length': 9.49, 'Number': 142},
        {'Diameter': 12, 'Length': 9.66, 'Number': 142},
        {'Diameter': 12, 'Length': 11.08, 'Number': 142},
        {'Diameter': 12, 'Length': 11.265, 'Number': 142},
        {'Diameter': 12, 'Length': 11.68, 'Number': 142},
        {'Diameter': 12, 'Length': 11.735, 'Number': 142},
        {'Diameter': 12, 'Length': 11.79, 'Number': 142},
        {'Diameter': 12, 'Length': 11.845, 'Number': 142},
        {'Diameter': 12, 'Length': 11.895, 'Number': 142},
        {'Diameter': 12, 'Length': 11.95, 'Number': 142},
        {'Diameter': 12, 'Length': 5.955, 'Number': 426},
        {'Diameter': 12, 'Length': 7.575, 'Number': 426},
        {'Diameter': 16, 'Length': 3.885, 'Number': 748},
        {'Diameter': 12, 'Length': 6.205, 'Number': 426},
        {'Diameter': 12, 'Length': 7.57, 'Number': 426},
        {'Diameter': 12, 'Length': 8.935, 'Number': 426},
        {'Diameter': 12, 'Length': 9.22, 'Number': 1278},
        {'Diameter': 12, 'Length': 8.87, 'Number': 576},
        {'Diameter': 12, 'Length': 8.77, 'Number': 1128},
        {'Diameter': 12, 'Length': 7.14, 'Number': 376},
        {'Diameter': 12, 'Length': 7.475, 'Number': 376},
        {'Diameter': 12, 'Length': 7.935, 'Number': 376},
        {'Diameter': 12, 'Length': 8.27, 'Number': 376},
        {'Diameter': 12, 'Length': 8.615, 'Number': 376},
        {'Diameter': 12, 'Length': 7.22, 'Number': 192},
        {'Diameter': 12, 'Length': 7.54, 'Number': 192},
        {'Diameter': 12, 'Length': 7.97, 'Number': 192},
        {'Diameter': 12, 'Length': 8.29, 'Number': 192},
        {'Diameter': 12, 'Length': 8.73, 'Number': 192}]

with open('LnT Kalyani Input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the rebar length and the number of pieces required
rebar_length = 12
total_original_length = sum(d['Length'] * d['Number'] for d in data)

# Define the solver
solver = pywraplp.Solver('CAB Rebar Optimization Model', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.IntVar(0, d['Number'], f'CutVar_{i}')

# Define the objective function
solver.Maximize(solver.Sum([cut_vars[i] for i in cut_vars]))

# Define the constraints
for i, d in enumerate(data):
    solver.Add(cut_vars[i] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    # Generate a new cutting pattern
    pattern = []
    for d in data:
        if d['Length'] < rebar_length and d['Number'] > 0:
            num_pieces_to_cut = min(d['Number'], int(rebar_length / d['Length']))
            pattern.append((d['Length'] * num_pieces_to_cut, d['Length'], num_pieces_to_cut))
            d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    # Solve the current LP relaxation
    solver.Solve()

    # Check if the solution is integer
    if all(cut_vars[i].solution_value() % 1 == 0 for i in cut_vars):
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j, (length, pattern_length, num_pieces) in enumerate(new_pattern):
            new_var = solver.IntVar(0, num_pieces, f'NewVar_{j}')
            new_vars.append(new_var)
            solver.Add(new_var == num_pieces)
            solver.Add(new_var * pattern_length <= rebar_length)
            solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.Add(new_var >= 0)
            solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})] >= new_var)

# Calculate percentage yield
total_cut_length = sum(cut_vars[i].solution_value() * data[i]['Length'] for i in cut_vars)
percentage_yield = (total_cut_length / total_original_length) * 100

# Print the results
print('Number of pieces required:', num_pieces)
print('Number of cuts:', solver.Objective().Value())
print('Percentage Yield:', percentage_yield)



In [1]:
import csv
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

# Define the input data
data = [{'Diameter': 12, 'Length': 12, 'Number': 3},
        {'Diameter': 12, 'Length': 5, 'Number': 7},
        {'Diameter': 12, 'Length': 6, 'Number': 5},
        {'Diameter': 12, 'Length': 6, 'Number': 4},
        {'Diameter': 12, 'Length': 7, 'Number': 2},
        {'Diameter': 12, 'Length': 8, 'Number': 8},
       ]

with open('Test_Data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the rebar length
rebar_length = 12

# Create a solver instance
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.IntVar(0, d['Number'], f'CutVar_{i}')

# Define the objective function: maximize total length of cut rebar
solver.Maximize(solver.Sum([cut_vars[i] * data[i]['Length'] for i in cut_vars]))

# Define constraints: each cut should be less than or equal to rebar length
for i, d in enumerate(data):
    solver.Add(cut_vars[i] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    pattern = []
    for d in data:
        if d['Length'] < rebar_length and d['Number'] > 0:
            num_pieces_to_cut = min(d['Number'], int(rebar_length / d['Length']))
            pattern.append((d['Length'] * num_pieces_to_cut, d['Length'], num_pieces_to_cut))
            d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    solver.Solve()

    # Check if the solution is integer
    if all(cut_vars[i].solution_value() % 1 == 0 for i in cut_vars):
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j, (length, pattern_length, num_pieces) in enumerate(new_pattern):
            new_var = solver.IntVar(0, num_pieces, f'NewVar_{j}')
            new_vars.append(new_var)
            solver.Add(new_var == num_pieces)
            solver.Add(new_var * pattern_length <= rebar_length)
            solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.Add(new_var >= 0)
            solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})] >= new_var)

# Calculate total yield
total_yield = sum(cut_vars[i].solution_value() * data[i]['Length'] for i in cut_vars)

# Print the results
print('Total Yield:', total_yield)
print('Percentage Yield:', total_yield / (rebar_length * sum(d['Number'] for d in data)) * 100)

# Print cutting patterns
print('Cutting Patterns:')
for i, d in enumerate(data):
    print(f"Diameter {d['Diameter']} - Length {d['Length']} - Number of Cuts: {cut_vars[i].solution_value()}")



In [19]:
import csv
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

# Define the input data
data = [{'Diameter': 12, 'Length': 12, 'Number': 3976},
        {'Diameter': 12, 'Length': 5.405, 'Number': 142},
        {'Diameter': 12, 'Length': 6, 'Number': 142},
        {'Diameter': 12, 'Length': 6.82, 'Number': 142},
        {'Diameter': 12, 'Length': 7.26, 'Number': 142},
        {'Diameter': 12, 'Length': 8.245, 'Number': 142},
        {'Diameter': 12, 'Length': 9.49, 'Number': 142},
        {'Diameter': 12, 'Length': 9.66, 'Number': 142},
        {'Diameter': 12, 'Length': 11.08, 'Number': 142},
        {'Diameter': 12, 'Length': 11.265, 'Number': 142},
        {'Diameter': 12, 'Length': 11.68, 'Number': 142},
        {'Diameter': 12, 'Length': 11.735, 'Number': 142},
        {'Diameter': 12, 'Length': 11.79, 'Number': 142},
        {'Diameter': 12, 'Length': 11.845, 'Number': 142},
        {'Diameter': 12, 'Length': 11.895, 'Number': 142},
        {'Diameter': 12, 'Length': 11.95, 'Number': 142},
        {'Diameter': 12, 'Length': 5.955, 'Number': 426},
        {'Diameter': 12, 'Length': 7.575, 'Number': 426},
        {'Diameter': 16, 'Length': 3.885, 'Number': 748},
        {'Diameter': 12, 'Length': 6.205, 'Number': 426},
        {'Diameter': 12, 'Length': 7.57, 'Number': 426},
        {'Diameter': 12, 'Length': 8.935, 'Number': 426},
        {'Diameter': 12, 'Length': 9.22, 'Number': 1278},
        {'Diameter': 12, 'Length': 8.87, 'Number': 576},
        {'Diameter': 12, 'Length': 8.77, 'Number': 1128},
        {'Diameter': 12, 'Length': 7.14, 'Number': 376},
        {'Diameter': 12, 'Length': 7.475, 'Number': 376},
        {'Diameter': 12, 'Length': 7.935, 'Number': 376},
        {'Diameter': 12, 'Length': 8.27, 'Number': 376},
        {'Diameter': 12, 'Length': 8.615, 'Number': 376},
        {'Diameter': 12, 'Length': 7.22, 'Number': 192},
        {'Diameter': 12, 'Length': 7.54, 'Number': 192},
        {'Diameter': 12, 'Length': 7.97, 'Number': 192},
        {'Diameter': 12, 'Length': 8.29, 'Number': 192},
        {'Diameter': 12, 'Length': 8.73, 'Number': 192}]

with open('LnT Kalyani Input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the rebar length
rebar_length = 12

# Create a solver instance
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = solver.IntVar(0, d['Number'], f'CutVar_{i}')

# Define the objective function: maximize total length of cut rebar
solver.Maximize(solver.Sum([cut_vars[i] * data[i]['Length'] for i in cut_vars]))

# Define constraints: each cut should be less than or equal to rebar length
for i, d in enumerate(data):
    solver.Add(cut_vars[i] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    pattern = []
    for d in data:
        if d['Length'] < rebar_length and d['Number'] > 0:
            num_pieces_to_cut = min(d['Number'], int(rebar_length / d['Length']))
            pattern.append((d['Length'] * num_pieces_to_cut, d['Length'], num_pieces_to_cut))
            d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    solver.Solve()

    # Check if the solution is integer
    if all(cut_vars[i].solution_value() % 1 == 0 for i in cut_vars):
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = []
        for j, (length, pattern_length, num_pieces) in enumerate(new_pattern):
            new_var = solver.IntVar(0, num_pieces, f'NewVar_{j}')
            new_vars.append(new_var)
            solver.Add(new_var == num_pieces)
            solver.Add(new_var * pattern_length <= rebar_length)
            solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
            solver.Add(new_var >= 0)
            solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})] >= new_var)

# Calculate total yield
total_yield = sum(cut_vars[i].solution_value() * data[i]['Length'] for i in cut_vars)

# Calculate percentage yield
percentage_yield = (total_yield / (rebar_length * sum(d['Number'] for d in data))) * 100

# Output cutting lengths
cutting_lengths = {d['Diameter']: d['Length'] for d in data}

# Print the results
print('Total Yield:', total_yield)
print('Percentage Yield:', percentage_yield)
print('Cutting Patterns:')
for i, d in enumerate(data):
    print(f"Diameter {d['Diameter']} - Length {cutting_lengths[d['Diameter']]} - Number of Cuts: {cut_vars[i].solution_value()}")